<a href="https://colab.research.google.com/github/samcast1/Short-Term-Investments-Model/blob/main/notebooks/3.1_sc_organizing_metrics_by_city.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Organizing Metrics By City
### Automated data transformations

I have several metrics taken over time for hundreds of cities - Each of these metrics will be involved in the multivariate time-series forecast, so I need to get each city in its own dataframe.

Dataframes will be indexed by date and contain 1 column for each metric.

Since I'm working with ~700 cities, I'll need to automate this somehow.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')

In [42]:
cut_price_shares = pd.read_csv('cut_price_shares.csv')
days_to_pending = pd.read_csv('days_to_pending.csv')
market_heat = pd.read_csv('market_heat.csv')
price_cut = pd.read_csv('price_cut.csv')
sales_to_list = pd.read_csv('sales_to_list.csv')
sales_values = pd.read_csv('sales_values.csv')
zhdf_bot = pd.read_csv('zhdf_bot.csv')
zhdf_top = pd.read_csv('zhdf_top.csv')
zhdf_mid = pd.read_csv('zhdf_mid.csv')
zvt1 = pd.read_csv('zvt1.csv')
zvt2 = pd.read_csv('zvt2.csv')
zvt3 = pd.read_csv('zvt3.csv')
zvt4 = pd.read_csv('zvt4.csv')
zvt5 = pd.read_csv('zvt5.csv')

In [12]:
print("cut_price_shares")
cut_price_shares.info()
print("days_to_pending")
days_to_pending.info()
print("market_heat")
market_heat.info()
print("price_cut")
price_cut.info()
print("sales_to_list")
sales_to_list.info()
print("sales_values")
sales_values.info()
print("zhdf_bot")
zhdf_bot.info()
print("zhdf_top")
zhdf_top.info()
print("zhdf_mid")
zhdf_mid.info()
print("zvt1")
zvt1.info()
print("zvt2")
zvt2.info()
print("zvt3")
zvt3.info()
print("zvt4")
zvt4.info()
print("zvt5")
zvt5.info()

cut_price_shares
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Columns: 927 entries, Unnamed: 0 to Lamesa, TX
dtypes: float64(926), object(1)
memory usage: 550.5+ KB
days_to_pending
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Columns: 767 entries, Unnamed: 0 to Craig, CO
dtypes: float64(766), object(1)
memory usage: 455.5+ KB
market_heat
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Columns: 929 entries, Unnamed: 0 to Lamesa, TX
dtypes: float64(928), object(1)
memory usage: 551.7+ KB
price_cut
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Columns: 703 entries, Unnamed: 0 to Ruidoso, NM
dtypes: float64(702), object(1)
memory usage: 417.5+ KB
sales_to_list
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 50 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  75 non-null     object 
 1   AL      

In [25]:
all_cities = [set(cut_price_shares.columns), set(days_to_pending.columns), set(market_heat.columns),\
              set(price_cut.columns), set(sales_values.columns), set(zhdf_bot.columns), set(zhdf_mid.columns),\
              set(zhdf_top.columns), set(zvt1.columns), set(zvt2.columns), set(zvt3.columns), set(zvt4.columns), set(zvt5.columns)]

In [26]:
common_cities = list(set.intersection(*all_cities))
common_cities.remove('Unnamed: 0')

In [27]:
len(common_cities)

509

In [28]:
sorted(common_cities)

['Aberdeen, WA',
 'Abilene, TX',
 'Adrian, MI',
 'Akron, OH',
 'Albany, NY',
 'Albany, OR',
 'Albemarle, NC',
 'Albuquerque, NM',
 'Alexandria, LA',
 'Allentown, PA',
 'Amarillo, TX',
 'Ames, IA',
 'Angola, IN',
 'Ann Arbor, MI',
 'Anniston, AL',
 'Appleton, WI',
 'Ardmore, OK',
 'Asheville, NC',
 'Ashland, OH',
 'Ashtabula, OH',
 'Astoria, OR',
 'Athens, GA',
 'Athens, TX',
 'Atlanta, GA',
 'Atlantic City, NJ',
 'Auburn, AL',
 'Augusta, GA',
 'Augusta, ME',
 'Austin, TX',
 'Bakersfield, CA',
 'Baltimore, MD',
 'Bangor, ME',
 'Baraboo, WI',
 'Barnstable Town, MA',
 'Bartlesville, OK',
 'Baton Rouge, LA',
 'Battle Creek, MI',
 'Bay City, MI',
 'Beaumont, TX',
 'Beckley, WV',
 'Bellefontaine, OH',
 'Bellingham, WA',
 'Bemidji, MN',
 'Bend, OR',
 'Big Rapids, MI',
 'Big Stone Gap, VA',
 'Billings, MT',
 'Binghamton, NY',
 'Birmingham, AL',
 'Bismarck, ND',
 'Blacksburg, VA',
 'Bloomington, IL',
 'Bloomington, IN',
 'Bloomsburg, PA',
 'Boise City, ID',
 'Boone, NC',
 'Boston, MA',
 'Boulde

In [43]:
def filter_city_columns(df):
  df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)
  df['date'] = pd.to_datetime(df['date'], format='mixed')
  df.set_index('date', inplace=True)

  df = df[df.columns.intersection(common_cities)]

  missed = []

  for city in common_cities:
    if city not in df.columns:
      missed.append(city)

  if len(missed) > 0:
    print(f"Missing the following cities: ", str(missed))

  else:
    print("All cities matching!")

  return df

In [44]:
 cut_price_shares = filter_city_columns(cut_price_shares)
 days_to_pending = filter_city_columns(days_to_pending)
 market_heat = filter_city_columns(market_heat)
 price_cut = filter_city_columns(price_cut)
 sales_values = filter_city_columns(sales_values)
 zhdf_bot = filter_city_columns(zhdf_bot)
 zhdf_top = filter_city_columns(zhdf_top)
 zhdf_mid = filter_city_columns(zhdf_mid)
 zvt1 = filter_city_columns(zvt1)
 zvt2 = filter_city_columns(zvt2)
 zvt3 = filter_city_columns(zvt3)
 zvt4 = filter_city_columns(zvt4)
 zvt5 = filter_city_columns(zvt5)

All cities matching!
All cities matching!
All cities matching!
All cities matching!
All cities matching!
All cities matching!
All cities matching!
All cities matching!
All cities matching!
All cities matching!
All cities matching!
All cities matching!
All cities matching!


In [53]:
sales_values

,United States,"New York, NY","Los Angeles, CA","Chicago, IL","Dallas, TX","Houston, TX","Washington, DC","Philadelphia, PA","Miami, FL","Atlanta, GA",...,"Scottsbluff, NE","Fremont, NE","Kill Devil Hills, NC","North Platte, NE","Brenham, TX","Angola, IN","Breckenridge, CO","Miami, OK","Toccoa, GA","Prineville, OR"
date,,,,,,,,,,,,,,,,,,,,,
2008-02-29,172500.0,399900.0,470000.0,218000.0,138000.0,145100.0,338000.0,191000.0,239000.0,150000.0,...,NaN,NaN,297500.0,NaN,NaN,NaN,395000.0,NaN,NaN,NaN
2008-03-31,177500.0,390000.0,455000.0,221575.0,145900.0,145000.0,327250.0,198900.0,235000.0,154500.0,...,NaN,NaN,305000.0,NaN,NaN,NaN,454000.0,NaN,NaN,NaN
2008-04-30,179000.0,390390.0,458000.0,221796.0,144900.0,146000.0,330000.0,200000.0,238000.0,154700.0,...,NaN,NaN,320000.0,NaN,NaN,NaN,445000.0,NaN,NaN,NaN
2008-05-31,180000.0,395000.0,440000.0,228000.0,150000.0,150000.0,335000.0,212000.0,232000.0,158950.0,...,NaN,NaN,325000.0,NaN,NaN,NaN,420000.0,NaN,NaN,NaN
2008-06-30,185000.0,400000.0,435000.0,235000.0,155500.0,156000.0,335000.0,218000.0,237600.0,158800.0,...,NaN,NaN,308500.0,NaN,NaN,NaN,448700.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,325000.0,565000.0,900000.0,285000.0,373000.0,315000.0,499000.0,315000.0,432000.0,345000.0,...,NaN,201000.0,570000.0,140000.0,NaN,289000.0,1005000.0,NaN,198000.0,NaN
2023-12-31,322500.0,561000.0,870750.0,280000.0,367372.0,310000.0,490000.0,311000.0,440000.0,336000.0,...,NaN,216250.0,680000.0,195500.0,NaN,221796.0,1012500.0,NaN,170000.0,NaN
2024-01-31,325000.0,565000.0,865000.0,280000.0,367350.0,307000.0,470000.0,300000.0,435000.0,330000.0,...,NaN,182500.0,600000.0,165000.0,NaN,205547.0,740000.0,NaN,165000.0,NaN


In [115]:
def save_city_metrics(idx):

  df = sales_values.iloc[-75:, idx]

  df = pd.concat([df, cut_price_shares.iloc[:,idx]], axis=1)
  df = pd.concat([df, days_to_pending.iloc[:,idx]], axis=1)
  df = pd.concat([df, market_heat.iloc[:,idx]], axis=1)
  df = pd.concat([df, price_cut.iloc[:,idx]], axis=1)

  df = pd.concat([df, zhdf_bot.iloc[-76:,idx]], axis=1)
  df = pd.concat([df, zhdf_mid.iloc[-76:,idx]], axis=1)
  df = pd.concat([df, zhdf_top.iloc[-76:,idx]], axis=1)
  df = pd.concat([df, zvt1.iloc[-77:,idx]], axis=1)
  df = pd.concat([df, zvt2.iloc[-77:,idx]], axis=1)
  df = pd.concat([df, zvt3.iloc[-77:,idx]], axis=1)
  df = pd.concat([df, zvt4.iloc[-77:,idx]], axis=1)
  df = pd.concat([df, zvt5.iloc[-77:,idx]], axis=1)


  column_names = ['sales_values','cut_price_shares','days_to_pending','market_heat','price_cut', 'zhdf_bot', 'zhdf_mid', 'zhdf_top', 'zvt1', 'zvt2', 'zvt3', 'zvt4', 'zvt5']
  df.columns = column_names

  city_name = cut_price_shares.columns[idx].replace(", ", "-").replace(" ", "").replace(".","").replace('-¦', 'n').lower()
  df.to_csv(f'city_metrics_2/{city_name}_metrics.csv', index=True)
  print(f'Saved {city_name} metrics')
  return df

In [116]:
for i in range(len(common_cities)):
  df = save_city_metrics(i)

Saved unitedstates metrics
Saved newyork-ny metrics
Saved losangeles-ca metrics
Saved chicago-il metrics
Saved dallas-tx metrics
Saved houston-tx metrics
Saved washington-dc metrics
Saved philadelphia-pa metrics
Saved miami-fl metrics
Saved atlanta-ga metrics
Saved boston-ma metrics
Saved phoenix-az metrics
Saved sanfrancisco-ca metrics
Saved riverside-ca metrics
Saved detroit-mi metrics
Saved seattle-wa metrics
Saved minneapolis-mn metrics
Saved sandiego-ca metrics
Saved tampa-fl metrics
Saved denver-co metrics
Saved baltimore-md metrics
Saved stlouis-mo metrics
Saved orlando-fl metrics
Saved charlotte-nc metrics
Saved sanantonio-tx metrics
Saved portland-or metrics
Saved sacramento-ca metrics
Saved pittsburgh-pa metrics
Saved cincinnati-oh metrics
Saved austin-tx metrics
Saved lasvegas-nv metrics
Saved kansascity-mo metrics
Saved columbus-oh metrics
Saved indianapolis-in metrics
Saved cleveland-oh metrics
Saved sanjose-ca metrics
Saved nashville-tn metrics
Saved virginiabeach-va metr


This data is almost ready, but I need to add the sales to list data, which is organized by state instead of city.

In [91]:
sales_to_list

,Unnamed: 0,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,2018-01-31,0.973026,0.974500,0.975310,0.995232,0.991388,0.971217,0.985771,0.965779,0.978232,...,0.983807,0.972172,0.973343,0.990170,0.982654,NaN,0.991411,0.979425,0.969691,0.996063
1,2018-02-28,0.976366,0.972710,0.981197,0.997403,0.993109,0.970068,0.973206,0.968798,0.975542,...,0.979381,0.973165,0.978543,0.990959,0.987141,NaN,0.992882,0.983908,0.969977,1.000000
2,2018-03-31,0.980754,0.979897,0.983725,1.000017,0.996926,0.973234,0.975674,0.971176,0.979092,...,0.981030,0.978066,0.983998,0.992647,0.988936,NaN,0.998972,0.992666,0.967764,1.000000
3,2018-04-30,0.986284,0.980898,0.982381,1.000998,0.997602,0.976208,0.974315,0.971488,0.978387,...,0.984962,0.982652,0.986519,0.992766,0.990091,NaN,0.999631,0.997509,0.964423,1.000000
4,2018-05-31,0.983696,0.979413,0.985022,0.999937,0.997888,0.978548,0.987668,0.973523,0.981614,...,0.987747,0.981954,0.985405,0.992051,0.992521,NaN,1.001122,1.000080,0.964286,0.997212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2023-11-30,0.982595,0.975823,0.982199,0.995849,0.988147,1.007997,0.992845,0.973371,0.990083,...,0.993689,0.979792,0.979522,0.986116,0.996671,1.003810,0.991303,1.000000,0.971460,0.973833
71,2023-12-31,0.979914,0.983080,0.982363,0.991588,0.983208,1.004903,0.991427,0.971419,0.987363,...,0.981845,0.977592,0.973383,0.984875,0.991563,1.000000,0.989669,0.997506,0.973473,0.975011
72,2024-01-31,0.979376,0.975026,0.980264,0.994182,0.984198,1.000701,0.989130,0.967882,0.986412,...,0.983022,0.977089,0.975152,0.983773,0.990398,0.996999,0.994027,0.993989,0.961698,0.976585
73,2024-02-29,0.982972,0.983369,0.980733,0.997398,0.985470,1.002922,0.991549,0.968467,0.989845,...,0.993446,0.980146,0.978564,0.990308,0.993610,0.997058,0.994948,1.000767,0.961296,0.965448


In [100]:
common_cities[2:5]

['Huntsville, TX', 'Richmond, KY', 'Indianapolis, IN']

In [118]:
for city in common_cities:
  if city != 'United States':
    city = city.replace(", ", "-").replace(" ", "").replace(".","").replace('-¦', 'n').lower()
    print(city)
    state = city.split('-')[1].upper()
    print(state)
    df = pd.read_csv(f'city_metrics_2/{city}_metrics.csv')
    if state == 'DC':
      state = 'VA'
    df['sales_to_list'] = sales_to_list[state]
    df.to_csv(f'city_metrics_3/{city}_metrics.csv', index=True)
    print(f'Saved {city} metrics')
  else:
    city = city.replace(" ", "").lower()
    df.to_csv(f'city_metrics_3/{city}_metrics.csv', index=True)
    print(f'Saved {city} metrics')





batonrouge-la
LA
Saved batonrouge-la metrics
muncie-in
IN
Saved muncie-in metrics
huntsville-tx
TX
Saved huntsville-tx metrics
richmond-ky
KY
Saved richmond-ky metrics
indianapolis-in
IN
Saved indianapolis-in metrics
zanesville-oh
OH
Saved zanesville-oh metrics
fairmont-wv
WV
Saved fairmont-wv metrics
poncacity-ok
OK
Saved poncacity-ok metrics
coosbay-or
OR
Saved coosbay-or metrics
manhattan-ks
KS
Saved manhattan-ks metrics
elkhart-in
IN
Saved elkhart-in metrics
mankato-mn
MN
Saved mankato-mn metrics
bowlinggreen-ky
KY
Saved bowlinggreen-ky metrics
winchester-va
VA
Saved winchester-va metrics
ashland-oh
OH
Saved ashland-oh metrics
rochester-mn
MN
Saved rochester-mn metrics
wilmington-nc
NC
Saved wilmington-nc metrics
columbus-ga
GA
Saved columbus-ga metrics
gardnervilleranchos-nv
NV
Saved gardnervilleranchos-nv metrics
nashville-tn
TN
Saved nashville-tn metrics
scottsbluff-ne
NE
Saved scottsbluff-ne metrics
atlanticcity-nj
NJ
Saved atlanticcity-nj metrics
tucson-az
AZ
Saved tucson-az m

In [119]:
import shutil
from google.colab import files

# Define the source and destination paths
source_folder = '/content/city_metrics_3'
zip_file = '/content/city_metrics.zip'

# Create a zip file containing all the files in the content folder
shutil.make_archive(zip_file.replace('.zip', ''), 'zip', source_folder)


'/content/city_metrics.zip'

In [120]:
files.download(zip_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>